In [1]:
import pandas as pd
import numpy as np
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
#from textblob import TexBlob
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report


# Get data news API 

In [3]:
from urllib.request import Request, urlopen
import json

import pandas as pd    

request=Request('https://stocknewsapi.com/api/v1?tickers=SNY&items=50&token=8gzujl...')
response = urlopen(request)
news = response.read()
news_data = json.loads(news)
df = pd.json_normalize(news_data['data'])

In [5]:
df = df[['date', 'title', 'sentiment']]
df.head()

,date,title,sentiment
0,"Mon, 14 Sep 2020 09:51:00 -0400",Sanofi's Dupixent Gets Breakthrough Status for...,Positive
1,"Fri, 11 Sep 2020 12:00:00 -0400",Sanofi : Information concerning the total numb...,Neutral
2,"Fri, 11 Sep 2020 11:30:14 -0400",Has Sanofi (SNY) Outpaced Other Medical Stocks...,Positive
3,"Fri, 11 Sep 2020 09:16:00 -0400",7 Solid Low Price-to-Sales Picks With Room for...,Positive
4,"Fri, 11 Sep 2020 07:04:00 -0400",Big Pharma wages stealth war on drug price wat...,Negative


In [6]:
df['date'] = df['date'].str.extract(r'(\d\d\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{4})')

In [7]:
df['date'] = pd.to_datetime(df.date).dt.strftime('%d-%m-%Y')

In [8]:
df['date'] = pd.to_datetime(df.date).dt.strftime('%Y-%m-%d')

In [9]:
df.date.unique()

array(['2020-09-14', '2020-11-09', '2020-09-09', '2020-08-09',
       '2020-07-09', '2020-05-09', '2020-04-09', '2020-03-09',
       '2020-01-09', '2020-08-30', '2020-08-28', '2020-08-26',
       '2020-08-24', '2020-08-19', '2020-08-17'], dtype=object)

In [10]:
df['sentiment'] = df.sentiment.replace({'Neutral':0, 'Positive':1, 'Negative':-1})

In [11]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

results = []

for headline in df['title']:
    pol_score = SIA().polarity_scores(headline) # run analysis
    pol_score['headline'] = headline # add headlines for viewing
    results.append(pol_score)

results[0]

{'neg': 0.0,
 'neu': 1.0,
 'pos': 0.0,
 'compound': 0.0,
 'headline': "Sanofi's Dupixent Gets Breakthrough Status for New Indication"}

We concatenate this list to our original dataframe

In [11]:
df['Compound'] = pd.DataFrame(results)['compound']
df['positive'] = pd.DataFrame(results)['pos']
df['Negative'] = pd.DataFrame(results)['neg']
df['Neutral'] = pd.DataFrame(results)['neu']

In [12]:
df.head()

,date,title,sentiment,Compound,positive,Negative,Neutral
0,2020-09-14,Sanofi's Dupixent Gets Breakthrough Status for...,1,0.0000,0.000,0.000,1.000
1,2020-11-09,Sanofi : Information concerning the total numb...,0,0.3612,0.241,0.000,0.759
2,2020-11-09,Has Sanofi (SNY) Outpaced Other Medical Stocks...,1,0.0000,0.000,0.000,1.000
3,2020-11-09,7 Solid Low Price-to-Sales Picks With Room for...,1,0.2732,0.372,0.186,0.442
4,2020-11-09,Big Pharma wages stealth war on drug price wat...,-1,-0.6369,0.000,0.426,0.574


We need to get the stock prices for SANOFI. We will get it from Yahoo Finance.
the main goal is to predict if the price increase or decrease

In [13]:
data = pd.read_csv("history_stock_Sanofi.csv")
data = data.drop(["Unnamed: 0", "Dividends"], axis=1)

In [14]:
data = data.rename(columns = {'Date': 'date'})
data.head()

,date,Open,High,Low,Close,Volume
0,2010-09-30,22.62,22.68,22.22,22.34,1915100
1,2010-10-01,22.42,22.43,22.09,22.26,3272400
2,2010-10-04,22.18,22.32,22.00,22.09,1727100
3,2010-10-05,22.75,23.12,22.68,23.03,2394700
4,2010-10-06,23.12,23.14,22.84,22.95,1695800


We create a new dateframe contains the stocke pricee and the news 

In [15]:
concat= df.merge(data, how = 'inner', on = 'date', left_index = True)

In [16]:
concat.head()

,date,title,sentiment,Compound,positive,Negative,Neutral,Open,High,Low,Close,Volume
2459,2020-07-09,"15 High-Yield, Low Volatility Blue-Chips For T...",1,-0.2732,0.000,0.174,0.826,51.11,51.29,50.21,50.55,795000
2459,2020-07-09,"15 High-Yield, Low Volatility Blue-Chips For T...",1,0.0000,0.130,0.130,0.741,51.11,51.29,50.21,50.55,795000
2397,2020-04-09,"Moderna, Pfizer, Johnson & Johnson, And Others...",0,-0.2960,0.000,0.128,0.872,43.31,43.87,41.27,43.83,4459000
2397,2020-04-09,Scott Gottlieb discusses vaccine timeline: Thi...,0,0.4767,0.237,0.000,0.763,43.31,43.87,41.27,43.83,4459000
2397,2020-04-09,Vaccine Hopes Rise Again: 4 Stocks to Watch,1,0.4215,0.318,0.000,0.682,43.31,43.87,41.27,43.83,4459000


# Models

We only have to select the numercis values  to make the predictions 

In [18]:
df_final = concat.drop(['date', 'title', 'Close'], axis=1)
df_final.head()

,sentiment,Compound,positive,Negative,Neutral,Open,High,Low,Volume
2459,1,-0.2732,0.000,0.174,0.826,51.11,51.29,50.21,795000
2459,1,0.0000,0.130,0.130,0.741,51.11,51.29,50.21,795000
2397,0,-0.2960,0.000,0.128,0.872,43.31,43.87,41.27,4459000
2397,0,0.4767,0.237,0.000,0.763,43.31,43.87,41.27,4459000
2397,1,0.4215,0.318,0.000,0.682,43.31,43.87,41.27,4459000


In [19]:
X = df_final.drop('sentiment', axis=1)
y = df_final.sentiment

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)

In [21]:
model = LinearDiscriminantAnalysis().fit(X_train, y_train)

In [22]:
predictions = model.predict(X_test)
predictions 

array([ 0,  1, -1, -1,  1,  0,  0,  0,  0,  0, -1], dtype=int64)

In [23]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

          -1       0.33      1.00      0.50         1
           0       0.67      0.67      0.67         6
           1       1.00      0.50      0.67         4

    accuracy                           0.64        11
   macro avg       0.67      0.72      0.61        11
weighted avg       0.76      0.64      0.65        11



In [24]:
results =  pd.DataFrame({'predictions':predictions, 'sentiment':y_test})

In [25]:
results.head()

,predictions,sentiment
2486,0,0
2495,1,1
2334,-1,-1
2486,-1,0
2493,1,1


We were able to recover that 50 data vi api, which explains errors in the predictions. With more data, the prediction will be better with fewer errors. 